In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChains
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.034, 0.034, 0.033, 0.033) seconds, 0.13 seconds total
Sampling took (0.053, 0.051, 0.045, 0.046) seconds, 0.19 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -892  2.6e-02     1.0  -894  -891  -891  1.6e+03  8.1e+03  1.0e+00
accept_stat__   0.91  2.5e-03    0.11  0.68  0.95   1.0  1.9e+03  1.0e+04  1.0e+00
stepsize__      0.84  3.4e-02   0.048  0.76  0.87  0.88  2.0e+00  1.0e+01  3.3e+13
treedepth__      1.9  8.0e-02    0.58   1.0   2.0   3.0  5.3e+01  2.7e+02  1.0e+00
n_leapfrog__     4.1  4.9e-01     3.2   1.0   3.0    11  4.2e+01  2.2e+02  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00      nan      nan      nan
energy__         893  3.6e-02     1.4   891   892   895  1.6e+03  8.1e+03  1.0e+00
sigma            7.8  5.7e-03    0.31   7.3   7.

end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*